In [8]:
import pandas 

kinetics = pandas.read_csv( 'kinetics.csv' )
thermal = pandas.read_csv( 'thermal.csv' ) 

print( kinetics.columns )
print( thermal.columns ) 

Index(['well', 'rate', 'mutant', 'protein_yield', 'dilution', 'note'], dtype='object')
Index(['well', 'rate', 'mutant'], dtype='object')


In [9]:
# for thermal 
# convert well indexes into temperatures based on map 

temp_key = pandas.read_csv( 'temperature_key.csv', index_col='Well' )
thermal['temp'] = thermal.well.str[0].map( temp_key.Celsius.to_dict() ) 

print( thermal.sample( 3 ) )

     well      rate mutant   temp
1922   C4 -0.000034   S16N  44.30
2606  G11  0.000190  W399S  32.90
431    H3  0.000291  E222R  30.05


In [10]:
# for kinetics, convert the rate into units (1/min) 
# hard-coded values:
#  0.0002 is the conversion from OD to M
#  0.25 is because we dilute the protein 4-fold upon initiation 
#  113330 is the extinction coef of BglB, used to convert mg/mL to M

kinetics[ 'kobs' ] = kinetics['rate'] * 0.0002 / ( kinetics[ 'protein_yield' ] * kinetics[ 'dilution' ] * 0.25 / 113300 )

print( kinetics.sample( 3 ) ) 

    well      rate   mutant  protein_yield  dilution  note       kobs
209   B3  0.007200    N404C           1.64      0.01   NaN  39.793171
550   G9  0.000012  T296A_2           0.21      0.10   NaN   0.051794
464  A11  0.000550    F415N           0.75      0.10   NaN   0.664693


In [11]:
# also for kinetics, convert well indexes into substrate concentrations

def my_s( x ):
    my_ss = dict( zip( 'ABCDEFGH', [ .75, .1875, .046875, .01171875, .002929688, .000732422, .000183105, 0 ] ) )
    return my_ss[ x[0] ] 

kinetics[ 's' ] = kinetics.well.map( my_s ) 

In [12]:
import numpy as np 
np.mean( [ .75, .1875, .046875, .01171875, .002929688, .000732422, .000183105, 0 ] )

0.124992370625

In [13]:
# anything we need to manually drop? 

to_drop = [ 'WT', 'WT1' ] 

for df in [ kinetics, thermal ]:
    for item in to_drop:
        if item in df.index:
            df.drop( item, inplace=True ) 

In [14]:
# now export "cleaned" data 

kinetics.to_csv( '../clean/kinetics.csv' ) 
thermal.to_csv( '../clean/thermal.csv' ) 